In [1]:
import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder \
    .appName('Lab4') \
    .config('spark.sql.legacy.timeParserPolicy', 'LEGACY') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 20:17:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Spark server is accessible here (click on Spark UI):

In [3]:
spark

In [4]:
def display_head(df: pyspark.sql.DataFrame, n: int = 10):
    return df.limit(n).toPandas().head(n)

## Zadanie 3

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType
from datetime import datetime

In [6]:
strptime = datetime.strptime
date_format = '%Y-%m-%d'
transactions_data = [
    ( 1, strptime('2011-01-01', date_format).date(), 500),
    ( 1, strptime('2011-01-15', date_format).date(), 50),
    ( 1, strptime('2011-01-22', date_format).date(), 250),
    ( 1, strptime('2011-01-24', date_format).date(), 75),
    ( 1, strptime('2011-01-26', date_format).date(), 125),
    ( 1, strptime('2011-01-28', date_format).date(), 175),
    ( 2, strptime('2011-01-01', date_format).date(), 500),
    ( 2, strptime('2011-01-15', date_format).date(), 50),
    ( 2, strptime('2011-01-22', date_format).date(), 25),
    ( 2, strptime('2011-01-23', date_format).date(), 125),
    ( 2, strptime('2011-01-26', date_format).date(), 200),
    ( 2, strptime('2011-01-29', date_format).date(), 250),
    ( 3, strptime('2011-01-01', date_format).date(), 500),
    ( 3, strptime('2011-01-15', date_format).date(), 50 ),
    ( 3, strptime('2011-01-22', date_format).date(), 5000),
    ( 3, strptime('2011-01-25', date_format).date(), 550),
    ( 3, strptime('2011-01-27', date_format).date(), 95 ),
    ( 3, strptime('2011-01-30', date_format).date(), 2500)
]

transaction_schema = StructType([
    StructField('account_id', IntegerType()),
    StructField('transaction_date', DateType()),
    StructField('transaction_amount', IntegerType())
])

transactions_df = spark.createDataFrame(transactions_data, transaction_schema)

In [7]:
display_head(transactions_df, 10)

,account_id,transaction_date,transaction_amount
0,1,2011-01-01,500
1,1,2011-01-15,50
2,1,2011-01-22,250
3,1,2011-01-24,75
4,1,2011-01-26,125
5,1,2011-01-28,175
6,2,2011-01-01,500
7,2,2011-01-15,50
8,2,2011-01-22,25
9,2,2011-01-23,125


In [8]:
logical_data = [
    (1,'George', 800),
    (2,'Sam', 950),
    (3,'Diane', 1100),
    (4,'Nicholas', 1250),
    (5,'Samuel', 1250),
    (6,'Patricia', 1300),
    (7,'Brian', 1500),
    (8,'Thomas', 1600),
    (9,'Fran', 2450),
    (10,'Debbie', 2850),
    (11,'Mark', 2975),
    (12,'James', 3000),
    (13,'Cynthia', 3000),
    (14,'Christopher', 5000)
]

logical_schema = StructType([
    StructField('row_id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('salary', IntegerType())
])

logical_df = spark.createDataFrame(logical_data, logical_schema)

In [9]:
display_head(logical_df, 10)

,row_id,first_name,salary
0,1,George,800
1,2,Sam,950
2,3,Diane,1100
3,4,Nicholas,1250
4,5,Samuel,1250
5,6,Patricia,1300
6,7,Brian,1500
7,8,Thomas,1600
8,9,Fran,2450
9,10,Debbie,2850


In [10]:
from pyspark.sql import Window
from pyspark.sql.functions import sum, mean, count, min, max

In [11]:
window_spec = Window.partitionBy('account_id').orderBy('transaction_date')

In [12]:
df = transactions_df \
    .withColumn('running_total_amount', sum('transaction_amount').over(window_spec)) \
    .orderBy('account_id', 'transaction_date')

In [13]:
display_head(df, 20)

,account_id,transaction_date,transaction_amount,running_total_amount
0,1,2011-01-01,500,500
1,1,2011-01-15,50,550
2,1,2011-01-22,250,800
3,1,2011-01-24,75,875
4,1,2011-01-26,125,1000
5,1,2011-01-28,175,1175
6,2,2011-01-01,500,500
7,2,2011-01-15,50,550
8,2,2011-01-22,25,575
9,2,2011-01-23,125,700


In [14]:
df = transactions_df \
    .withColumn('running_mean_amount', mean('transaction_amount').over(window_spec)) \
    .withColumn('running_transaction_quantity', count('*').over(window_spec)) \
    .withColumn('running_min_amount', min('transaction_amount').over(window_spec)) \
    .withColumn('running_max_amount', max('transaction_amount').over(window_spec)) \
    .withColumn('running_total_amount', sum('transaction_amount').over(window_spec)) \
    .orderBy('account_id', 'transaction_date')

In [15]:
display_head(df, 20)

,account_id,transaction_date,transaction_amount,running_mean_amount,running_transaction_quantity,running_min_amount,running_max_amount,running_total_amount
0,1,2011-01-01,500,500.000000,1,500,500,500
1,1,2011-01-15,50,275.000000,2,50,500,550
2,1,2011-01-22,250,266.666667,3,50,500,800
3,1,2011-01-24,75,218.750000,4,50,500,875
4,1,2011-01-26,125,200.000000,5,50,500,1000
5,1,2011-01-28,175,195.833333,6,50,500,1175
6,2,2011-01-01,500,500.000000,1,500,500,500
7,2,2011-01-15,50,275.000000,2,50,500,550
8,2,2011-01-22,25,191.666667,3,25,500,575
9,2,2011-01-23,125,175.000000,4,25,500,700


In [16]:
window_spec = Window.partitionBy('account_id').orderBy('transaction_date').rowsBetween(-2, 0)

In [17]:
df = transactions_df \
    .withColumn('sliding_mean', mean('transaction_amount').over(window_spec)) \
    .withColumn('sliding_count', count('*').over(window_spec)) \
    .withColumn('sliding_min', min('transaction_amount').over(window_spec)) \
    .withColumn('sliding_max', max('transaction_amount').over(window_spec)) \
    .withColumn('sliding_total', sum('transaction_amount').over(window_spec)) \
    .orderBy('account_id', 'transaction_date')

In [18]:
display_head(df, 20)

,account_id,transaction_date,transaction_amount,sliding_mean,sliding_count,sliding_min,sliding_max,sliding_total
0,1,2011-01-01,500,500.000000,1,500,500,500
1,1,2011-01-15,50,275.000000,2,50,500,550
2,1,2011-01-22,250,266.666667,3,50,500,800
3,1,2011-01-24,75,125.000000,3,50,250,375
4,1,2011-01-26,125,150.000000,3,75,250,450
5,1,2011-01-28,175,125.000000,3,75,175,375
6,2,2011-01-01,500,500.000000,1,500,500,500
7,2,2011-01-15,50,275.000000,2,50,500,550
8,2,2011-01-22,25,191.666667,3,25,500,575
9,2,2011-01-23,125,66.666667,3,25,125,200


In [19]:
window_rows = Window.orderBy('salary').rowsBetween(Window.unboundedPreceding, Window.currentRow)

window_range = Window.orderBy('salary').rangeBetween(Window.unboundedPreceding, Window.currentRow)

In [20]:
df = logical_df \
    .withColumn('sum_by_rows', sum('salary').over(window_rows)) \
    .withColumn('sum_by_range', sum('salary').over(window_range))

In [21]:
display_head(df, 20)

23/04/10 20:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/10 20:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/10 20:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/10 20:24:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/04/10 20:24:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,row_id,first_name,salary,sum_by_rows,sum_by_range
0,1,George,800,800,800
1,2,Sam,950,1750,1750
2,3,Diane,1100,2850,2850
3,4,Nicholas,1250,4100,5350
4,5,Samuel,1250,5350,5350
5,6,Patricia,1300,6650,6650
6,7,Brian,1500,8150,8150
7,8,Thomas,1600,9750,9750
8,9,Fran,2450,12200,12200
9,10,Debbie,2850,15050,15050


## Zadanie 4

In [23]:
from pyspark.sql.functions import lead, lag, row_number

In [24]:
window_spec = Window.partitionBy('account_id').orderBy('transaction_date')

In [25]:
df = transactions_df \
    .withColumn('next_amount', lead('transaction_amount').over(window_spec)) \
    .withColumn('previous_amount', lag('transaction_amount').over(window_spec)) \
    .withColumn('row_number', row_number().over(window_spec)) \
    .orderBy('account_id', 'transaction_date')

In [26]:
display_head(df, 20)

,account_id,transaction_date,transaction_amount,next_amount,previous_amount,row_number
0,1,2011-01-01,500,50.0,NaN,1
1,1,2011-01-15,50,250.0,500.0,2
2,1,2011-01-22,250,75.0,50.0,3
3,1,2011-01-24,75,125.0,250.0,4
4,1,2011-01-26,125,175.0,75.0,5
5,1,2011-01-28,175,NaN,125.0,6
6,2,2011-01-01,500,50.0,NaN,1
7,2,2011-01-15,50,25.0,500.0,2
8,2,2011-01-22,25,125.0,50.0,3
9,2,2011-01-23,125,200.0,25.0,4
